<a href="https://colab.research.google.com/github/pavankumarchakali-25/indian-music-recommender/blob/main/Music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')  # Run if not mounted
df.to_csv('/content/drive/MyDrive/indian_songs_spotify.csv', index=False)
print("Dataset saved to Google Drive.")

Mounted at /content/drive
Dataset saved to Google Drive.


In [20]:
from google.colab import files
files.download('/content/indian_songs_spotify.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
!pip install numpy==1.26.4

In [1]:
!pip install surprise spotipy pandas numpy librosa indic-nlp-library langdetect

In [2]:
import pandas as pd

# Simulated dataset: user_id, song_id, rating, language, name (Indian languages only)
data = {
    'user_id': [1, 1, 2, 2, 3, 3, 4, 4],
    'song_id': ['song1_hin', 'song2_tam', 'song1_hin', 'song3_tel', 'song2_tam', 'song4_hin', 'song3_tel', 'song5_pun'],
    'rating': [5, 3, 4, 2, 5, 4, 3, 5],
    'language': ['hi', 'ta', 'hi', 'te', 'ta', 'hi', 'te', 'pa'],
    'name': ['Tum Hi Ho', 'Vaathi Coming', 'Tum Hi Ho', 'Rangabati', 'Vaathi Coming', 'Dil Se', 'Rangabati', 'Mundian To Bach Ke']
}
df = pd.DataFrame(data)
df.to_csv('/content/indian_songs.csv', index=False)
print(df)

   user_id    song_id  rating language                name
0        1  song1_hin       5       hi           Tum Hi Ho
1        1  song2_tam       3       ta       Vaathi Coming
2        2  song1_hin       4       hi           Tum Hi Ho
3        2  song3_tel       2       te           Rangabati
4        3  song2_tam       5       ta       Vaathi Coming
5        3  song4_hin       4       hi              Dil Se
6        4  song3_tel       3       te           Rangabati
7        4  song5_pun       5       pa  Mundian To Bach Ke


In [3]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/indian_songs.csv')

# Prepare data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'song_id', 'rating']], reader)

# Split into train and test sets (80% train, 20% test)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Train SVD model
model = SVD()
model.fit(trainset)

# Evaluate model on test set
predictions = model.test(testset)
print(f"RMSE: {accuracy.rmse(predictions)}")

# Example: Recommend top 3 songs for user 1
user_id = 1
indian_songs = df['song_id'].unique()
pred_ratings = [model.predict(user_id, song_id) for song_id in indian_songs]
top_recommendations = sorted(pred_ratings, key=lambda x: x.est, reverse=True)[:3]

# Display recommendations
print(f"Top 3 recommendations for User {user_id}:")
for pred in top_recommendations:
    song_name = df[df['song_id'] == pred.iid]['name'].iloc[0]
    print(f"Song: {song_name}, Predicted Rating: {pred.est:.2f}")

RMSE: 1.9101
RMSE: 1.910146747707422
Top 3 recommendations for User 1:
Song: Tum Hi Ho, Predicted Rating: 4.45
Song: Mundian To Bach Ke, Predicted Rating: 4.36
Song: Dil Se, Predicted Rating: 4.32


In [4]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from surprise import SVD, Dataset, Reader

# Load the dataset and model (re-running to ensure everything is in memory)
df = pd.read_csv('/content/indian_songs.csv')
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'song_id', 'rating']], reader)
trainset = data.build_full_trainset()  # Use full dataset for training
model = SVD()
model.fit(trainset)

# Create widgets for user input
user_id = widgets.IntText(value=1, description='User ID:', style={'description_width': 'initial'})
button = widgets.Button(description='Get Recommendations', button_style='success')
output = widgets.Output()

# Function to handle button click
def on_button_clicked(b):
    with output:
        output.clear_output()
        indian_songs = df['song_id'].unique()
        pred_ratings = [model.predict(user_id.value, song_id) for song_id in indian_songs]
        top_recommendations = sorted(pred_ratings, key=lambda x: x.est, reverse=True)[:3]
        print(f"Top 3 recommendations for User {user_id.value}:")
        for pred in top_recommendations:
            song_name = df[df['song_id'] == pred.iid]['name'].iloc[0]
            print(f"Song: {song_name}, Predicted Rating: {pred.est:.2f}")

button.on_click(on_button_clicked)

# Display the interface
display(user_id, button, output)

IntText(value=1, description='User ID:', style=DescriptionStyle(description_width='initial'))

Button(button_style='success', description='Get Recommendations', style=ButtonStyle())

Output()

In [14]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import random

# Replace with your Spotify API credentials
client_id = '2612ff4ccd5549e194c505b04815d00e'  # Paste your Client ID here
client_secret = 'a154390a695c4739bb7670742ec47247'  # Paste your Client Secret here
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Expanded list of Indian artists
indian_artists = [
    'Arijit Singh',  # Hindi
    'Shreya Ghoshal',  # Hindi/Multiple
    'Anirudh Ravichander',  # Tamil
    'Ilaiyaraaja',  # Tamil
    'S. P. Balasubrahmanyam',  # Telugu
    'Diljit Dosanjh',  # Punjabi
    'A. R. Rahman',  # Tamil/Multiple
    'Sidhu Moose Wala',  # Punjabi
    'Vishal Dadlani',  # Hindi
    'Devi Sri Prasad',  # Telugu
    'Badshah',  # Punjabi/Hindi
    'Sunidhi Chauhan',  # Hindi
    'Harris Jayaraj',  # Tamil
    'Chinmayi',  # Tamil/Telugu
    'Yo Yo Honey Singh',  # Punjabi/Hindi
    'Neha Kakkar',  # Hindi
    'Guru Randhawa',  # Punjabi
    'Yuvan Shankar Raja'  # Tamil
]

# Language mapping for artists
artist_to_language = {
    'Arijit Singh': 'hi',
    'Shreya Ghoshal': 'hi',
    'Anirudh Ravichander': 'ta',
    'Ilaiyaraaja': 'ta',
    'S. P. Balasubrahmanyam': 'te',
    'Diljit Dosanjh': 'pa',
    'A. R. Rahman': 'ta',
    'Sidhu Moose Wala': 'pa',
    'Vishal Dadlani': 'hi',
    'Devi Sri Prasad': 'te',
    'Badshah': 'pa',
    'Sunidhi Chauhan': 'hi',
    'Harris Jayaraj': 'ta',
    'Chinmayi': 'ta',
    'Yo Yo Honey Singh': 'pa',
    'Neha Kakkar': 'hi',
    'Guru Randhawa': 'pa',
    'Yuvan Shankar Raja': 'ta'
}

# Fetch all tracks with pagination
tracks = []
for artist in indian_artists:
    try:
        offset = 0
        total_fetched = 0
        while True:
            results = sp.search(q=f'artist:{artist}', type='track', limit=50, offset=offset)  # Removed market:IN
            track_items = results['tracks']['items']
            if not track_items:
                break
            for track in track_items:
                tracks.append({
                    'song_id': track['id'],
                    'name': track['name'],
                    'artist': track['artists'][0]['name'],
                    'language': artist_to_language.get(artist, 'unknown'),
                    'popularity': track['popularity']
                })
            total_fetched += len(track_items)
            print(f"Fetched {len(track_items)} tracks for {artist} at offset {offset}, Total: {total_fetched}")
            offset += 50
    except Exception as e:
        print(f"Error fetching tracks for {artist}: {e}")
        continue

# Create DataFrame and remove duplicates
df = pd.DataFrame(tracks)
df = df.drop_duplicates(subset='song_id')  # Ensure unique songs
if df.empty:
    print("No tracks fetched. Check API credentials or internet connection.")
else:
    print(f"Total fetched: {len(df)} tracks across {len(df['song_id'].unique())} unique songs.")
    print(df.groupby('language').size())  # Show language distribution

# Simulate user ratings
users = [1, 2, 3, 4, 5, 6]  # Added more users
ratings_data = []
for user_id in users:
    if not df.empty:
        sampled_songs = df['song_id'].sample(min(30, len(df)), random_state=user_id)  # Increased to 30
        for song_id in sampled_songs:
            ratings_data.append({
                'user_id': user_id,
                'song_id': song_id,
                'rating': random.randint(1, 5),
                'name': df[df['song_id'] == song_id]['name'].iloc[0],
                'language': df[df['song_id'] == song_id]['language'].iloc[0]
            })

df_ratings = pd.DataFrame(ratings_data)
if df_ratings.empty:
    print("No ratings generated. Check if tracks were fetched.")
else:
    df_ratings.to_csv('/content/indian_songs_spotify.csv', index=False)
    print(df_ratings.head(10))

Fetched 49 tracks for Arijit Singh at offset 0, Total: 49
Fetched 50 tracks for Arijit Singh at offset 50, Total: 99
Fetched 50 tracks for Arijit Singh at offset 100, Total: 149
Fetched 50 tracks for Arijit Singh at offset 150, Total: 199
Fetched 50 tracks for Arijit Singh at offset 200, Total: 249
Fetched 50 tracks for Arijit Singh at offset 250, Total: 299
Fetched 50 tracks for Arijit Singh at offset 300, Total: 349
Fetched 50 tracks for Arijit Singh at offset 350, Total: 399
Fetched 50 tracks for Arijit Singh at offset 400, Total: 449
Fetched 50 tracks for Arijit Singh at offset 450, Total: 499
Fetched 50 tracks for Arijit Singh at offset 500, Total: 549
Fetched 50 tracks for Arijit Singh at offset 550, Total: 599
Fetched 50 tracks for Arijit Singh at offset 600, Total: 649
Fetched 50 tracks for Arijit Singh at offset 650, Total: 699
Fetched 50 tracks for Arijit Singh at offset 700, Total: 749
Fetched 50 tracks for Arijit Singh at offset 750, Total: 799
Fetched 50 tracks for Arijit 

ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Arijit Singh', 'limit': 50, 'offset': 1000, 'type': 'track', 'market': None} returned 400 due to Limit + Offset exceeds maximum of 1000


Fetched 50 tracks for Arijit Singh at offset 950, Total: 999
Error fetching tracks for Arijit Singh: http status: 400, code: -1 - https://api.spotify.com/v1/search?q=artist%3AArijit+Singh&limit=50&offset=1000&type=track:
 Limit + Offset exceeds maximum of 1000, reason: None
Fetched 49 tracks for Shreya Ghoshal at offset 0, Total: 49
Fetched 50 tracks for Shreya Ghoshal at offset 50, Total: 99
Fetched 50 tracks for Shreya Ghoshal at offset 100, Total: 149
Fetched 50 tracks for Shreya Ghoshal at offset 150, Total: 199
Fetched 50 tracks for Shreya Ghoshal at offset 200, Total: 249
Fetched 50 tracks for Shreya Ghoshal at offset 250, Total: 299
Fetched 50 tracks for Shreya Ghoshal at offset 300, Total: 349
Fetched 50 tracks for Shreya Ghoshal at offset 350, Total: 399
Fetched 50 tracks for Shreya Ghoshal at offset 400, Total: 449
Fetched 50 tracks for Shreya Ghoshal at offset 450, Total: 499
Fetched 50 tracks for Shreya Ghoshal at offset 500, Total: 549
Fetched 50 tracks for Shreya Ghoshal 

ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Shreya Ghoshal', 'limit': 50, 'offset': 1000, 'type': 'track', 'market': None} returned 400 due to Limit + Offset exceeds maximum of 1000


Fetched 50 tracks for Shreya Ghoshal at offset 950, Total: 999
Error fetching tracks for Shreya Ghoshal: http status: 400, code: -1 - https://api.spotify.com/v1/search?q=artist%3AShreya+Ghoshal&limit=50&offset=1000&type=track:
 Limit + Offset exceeds maximum of 1000, reason: None
Fetched 50 tracks for Anirudh Ravichander at offset 0, Total: 50
Fetched 50 tracks for Anirudh Ravichander at offset 50, Total: 100
Fetched 50 tracks for Anirudh Ravichander at offset 100, Total: 150
Fetched 50 tracks for Anirudh Ravichander at offset 150, Total: 200
Fetched 50 tracks for Anirudh Ravichander at offset 200, Total: 250
Fetched 50 tracks for Anirudh Ravichander at offset 250, Total: 300
Fetched 50 tracks for Anirudh Ravichander at offset 300, Total: 350
Fetched 50 tracks for Anirudh Ravichander at offset 350, Total: 400
Fetched 50 tracks for Anirudh Ravichander at offset 400, Total: 450
Fetched 50 tracks for Anirudh Ravichander at offset 450, Total: 500
Fetched 50 tracks for Anirudh Ravichander a

...

Fetched 50 tracks for Chinmayi at offset 950, Total: 1000
Error fetching tracks for Chinmayi: http status: 400, code: -1 - https://api.spotify.com/v1/search?q=artist%3AChinmayi&limit=50&offset=1000&type=track:
 Limit + Offset exceeds maximum of 1000, reason: None
Fetched 50 tracks for Yo Yo Honey Singh at offset 0, Total: 50
Fetched 50 tracks for Yo Yo Honey Singh at offset 50, Total: 100
Fetched 50 tracks for Yo Yo Honey Singh at offset 100, Total: 150
Fetched 50 tracks for Yo Yo Honey Singh at offset 150, Total: 200
Fetched 50 tracks for Yo Yo Honey Singh at offset 200, Total: 250
Fetched 50 tracks for Yo Yo Honey Singh at offset 250, Total: 300
Fetched 50 tracks for Yo Yo Honey Singh at offset 300, Total: 350
Fetched 50 tracks for Yo Yo Honey Singh at offset 350, Total: 400
Fetched 50 tracks for Yo Yo Honey Singh at offset 400, Total: 450
Fetched 50 tracks for Yo Yo Honey Singh at offset 450, Total: 500
Fetched 50 tracks for Yo Yo Honey Singh at offset 500, Total: 550
Fetched 50 tra

ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Neha Kakkar', 'limit': 50, 'offset': 1000, 'type': 'track', 'market': None} returned 400 due to Limit + Offset exceeds maximum of 1000


Fetched 50 tracks for Neha Kakkar at offset 950, Total: 1000
Error fetching tracks for Neha Kakkar: http status: 400, code: -1 - https://api.spotify.com/v1/search?q=artist%3ANeha+Kakkar&limit=50&offset=1000&type=track:
 Limit + Offset exceeds maximum of 1000, reason: None
Fetched 50 tracks for Guru Randhawa at offset 0, Total: 50
Fetched 50 tracks for Guru Randhawa at offset 50, Total: 100
Fetched 50 tracks for Guru Randhawa at offset 100, Total: 150
Fetched 50 tracks for Guru Randhawa at offset 150, Total: 200
Fetched 50 tracks for Guru Randhawa at offset 200, Total: 250
Fetched 50 tracks for Guru Randhawa at offset 250, Total: 300
Fetched 50 tracks for Guru Randhawa at offset 300, Total: 350
Fetched 50 tracks for Guru Randhawa at offset 350, Total: 400
Fetched 50 tracks for Guru Randhawa at offset 400, Total: 450
Fetched 50 tracks for Guru Randhawa at offset 450, Total: 500
Fetched 50 tracks for Guru Randhawa at offset 500, Total: 550
Fetched 14 tracks for Guru Randhawa at offset 550

ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Yuvan Shankar Raja', 'limit': 50, 'offset': 1000, 'type': 'track', 'market': None} returned 400 due to Limit + Offset exceeds maximum of 1000


Fetched 50 tracks for Yuvan Shankar Raja at offset 950, Total: 1000
Error fetching tracks for Yuvan Shankar Raja: http status: 400, code: -1 - https://api.spotify.com/v1/search?q=artist%3AYuvan+Shankar+Raja&limit=50&offset=1000&type=track:
 Limit + Offset exceeds maximum of 1000, reason: None
Total fetched: 14766 tracks across 14766 unique songs.
language
hi    4441
pa    3205
ta    5387
te    1733
dtype: int64
   user_id                 song_id  rating  \
0        1  20GvRntcVUf2iB9yuN5jZ3       1   
1        1  3sV46smfulDcKYfj4FdrjR       1   
2        1  6J6j4L2Jzy9VmwWj71ORPU       1   
3        1  6DzB9TCOLVGTtaXmJNEmCZ       5   
4        1  7nAaXFOtJtL8iee4xSxJLv       3   
5        1  0yLQV941yUzphlPNKx0icK       2   
6        1  26gUBt9LDMIkdxskiRpbZ0       2   
7        1  5eYDMV3Fn8KElacH754Bjt       2   
8        1  6CKNnPD33ZuXBJdlXVmzXD       2   
9        1  0xNJMvunwx1loBS9Zu64Z0       4   

                                             name language  
0                

In [18]:
import ipywidgets as widgets
from IPython.display import display
from surprise import SVD, Dataset, Reader
import pandas as pd

# Load the Spotify dataset
try:
    df = pd.read_csv('/content/indian_songs_spotify.csv')
except FileNotFoundError:
    print("Dataset file not found. Ensure the previous step created indian_songs_spotify.csv.")
    raise

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'song_id', 'rating']], reader)
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)

# Print dataset info
valid_users = df['user_id'].unique()
print(f"Dataset has {len(df['song_id'].unique())} unique songs and {len(valid_users)} unique users: {valid_users}")
print(f"Language distribution:\n{df.groupby('language').size()}")

# Create widgets
user_id = widgets.IntText(value=1, description='User ID:', style={'description_width': 'initial'})
button = widgets.Button(description='Get Recommendations', button_style='success')
output = widgets.Output()

# Function to handle button click
def on_button_clicked(b):
    with output:
        output.clear_output()
        if user_id.value not in valid_users:
            print(f"Invalid User ID: {user_id.value}. Please choose from {valid_users}.")
            return
        indian_songs = df['song_id'].unique()
        pred_ratings = [model.predict(user_id.value, song_id) for song_id in indian_songs]
        top_recommendations = sorted(pred_ratings, key=lambda x: x.est, reverse=True)[:5]
        print(f"Top 5 recommendations for User {user_id.value}:")
        for pred in top_recommendations:
            song_name = df[df['song_id'] == pred.iid]['name'].iloc[0]
            song_lang = df[df['song_id'] == pred.iid]['language'].iloc[0]
            print(f"Song: {song_name} ({song_lang}), Predicted Rating: {pred.est:.2f}")

button.on_click(on_button_clicked)
display(user_id, button, output)

Dataset has 180 unique songs and 6 unique users: [1 2 3 4 5 6]
Language distribution:
language
hi    53
pa    34
ta    76
te    17
dtype: int64


IntText(value=1, description='User ID:', style=DescriptionStyle(description_width='initial'))

Button(button_style='success', description='Get Recommendations', style=ButtonStyle())

Output()